# Gradient Boosting Trees

En este tutorial trabajaremos con los modelos basados en Gradiente Boosting Trees. A parte de trabajar con las implementaciones de *sklearn* `HistGradientBoosting`, veremos otras implementaciones de *XGBoost*, y *LightGBM*.

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
import multiprocessing

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

Trabajaremos con el dataset "adult" que intenta clasificar si una persona gana más de 50K$ al año en base a ciertas características.

In [ ]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml("adult", version=2, return_X_y=True)

# Quitamos dos columnas:
# - "education-num" porque es redundante con "education"
# - "fnlwgt" (peso final) porque no se sabe qué significa
X = X.drop(["education-num", "fnlwgt"], axis="columns")
X.dtypes

Observamos que tenemos variables categóricas y numéricas. En nuestras versiones de Gradient Boosting Trees no va a ser un problema

In [ ]:
X.describe(include="all")

Podemos comprobar que también existen "missing values".

In [ ]:
X.isnull().sum()

In [ ]:
X.head()

In [ ]:
y.value_counts().sort_index()

Es un problema desbalanceado, tomaremos algunas medidas.

In [ ]:
# Holdout para la evaluación del modelo. 33% de los datos disponibles para test
# Este sería la evalución outer loop
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, stratify=y, random_state=42)

# HistGradientBoostingClassifier
Los parámetros más importantes de la implantación de sklearn (`HistGradientBoostingClassifier`) para controlar el crecimiento de los árboles, la velocidad de aprendizaje del modelo, y los que gestionan la parada temprana para evitar *overfitting*, son:

- `learning_rate`: reduce la contribución de cada árbol multiplicando su influencia original por este valor.
- `max_iter`: El número máximo de iteraciones del proceso de boosting, es decir, el número máximo de árboles.
- `max_depth`: profundidad máxima que pueden alcanzar los árboles.
- `min_samples_split`: número mínimo de observaciones que debe de tener un nodo para que pueda dividirse. Si es un valor decimal se interpreta como fracción del total de observaciones de entrenamiento `ceil(min_samples_split * n_samples)`.
- `min_samples_leaf`: número mínimo de observaciones que debe de tener cada uno de los nodos hijos para que se produzca la división. Si es un valor decimal se interpreta como fracción del total de observaciones de entrenamiento `ceil(min_samples_split * n_samples)`.
- `validation_fraction`: proporción de datos separados del conjunto entrenamiento y empleados como conjunto de validación para determinar la parada temprana (*early stopping*).
- `n_iter_no_change`: número de iteraciones consecutivas en las que no se debe superar el tol para que el algoritmo se detenga (*early stopping*). Si su valor es None se desactiva la parada temprana.
- `tol`: porcentaje mínimo de mejora entre dos iteraciones consecutivas por debajo del cual se considera que el modelo no ha mejorado.


In [ ]:
# Creación del modelo (ATENCION: este modelo admite variables categóricas y numéricas)
# categorical_features="from_dtype" indica que las variables categóricas son las que 
# son de tipo object
# ==============================================================================
cls_gb = HistGradientBoostingClassifier(
            categorical_features="from_dtype",
            random_state = 42
         )

# Entrenamiento del modelo
# ==============================================================================
np.random.seed(42)
cls_gb.fit(X_train, y_train)

In [ ]:
# Evaluación del modelo
y_test_pred = cls_gb.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

## Búsqueda de parámetros con Random Search
Vamos a realizar una búsqueda usando Random Search

In [ ]:
param_grid = {'max_depth'         : [None, 1, 3, 5, 10, 20, 30],
              'max_iter'          : sp_randint(50, 500),
              'learning_rate'     : [0.001, 0.01, 0.1, 0.2],
              'l2_regularization' : [0, 1],
              'max_leaf_nodes'    : [3, 10, 30, 40]
             }

inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

budget = 20
# Cross-validation (3-fold) para la búsqueda de hiper-parámetros
clf = RandomizedSearchCV (estimator  = HistGradientBoostingClassifier(
                                max_iter            = 1000,
                                random_state        = 42,
                                categorical_features="from_dtype",
                                # Activación de la parada temprana
                                validation_fraction = 0.1,
                                n_iter_no_change    = 5,
                                tol                 = 0.0001),
                           param_distributions = param_grid,
                           scoring='balanced_accuracy',
                           cv=inner,
                           refit=True,
                           n_jobs=-1,
                           verbose=1,
                           n_iter=budget,
                           return_train_score=True)

np.random.seed(42)
clf.fit(X=X_train, y=y_train)


In [ ]:
# Resultados
# ==============================================================================
resultados = pd.DataFrame(clf.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

In [ ]:
clf.best_params_, clf.best_score_

In [ ]:
modelo_final = clf.best_estimator_
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

## Importancia por permutación
Identifica la influencia que tiene cada predictor sobre una determinada métrica de evaluación del modelo 

In [ ]:
from sklearn.inspection import permutation_importance
import multiprocessing

importancia = permutation_importance(
                estimator    = modelo_final,
                X            = X_train,
                y            = y_train,
                n_repeats    = 5,
                scoring      = 'accuracy',
                n_jobs       = multiprocessing.cpu_count() - 1,
                random_state = 42
             )

# Se almacenan los resultados (media y desviación) en un dataframe
df_importancia = pd.DataFrame(
                    {k: importancia[k] for k in ['importances_mean', 'importances_std']}
                 )

df_importancia['feature'] = X_train.columns
df_importancia.sort_values('importances_mean', ascending=False)

In [ ]:
# Gráfico
fig, ax = plt.subplots(figsize=(5, 6))
df_importancia = df_importancia.sort_values('importances_mean', ascending=True)
ax.barh(
    df_importancia['feature'],
    df_importancia['importances_mean'],
    xerr=df_importancia['importances_std'],
    align='center',
    alpha=0
)
ax.plot(
    df_importancia['importances_mean'],
    df_importancia['feature'],
    marker="D",
    linestyle="",
    alpha=0.8,
    color="r"
)
ax.set_title('Importancia de los predictores (train)')
ax.set_xlabel('Incremento del error tras la permutación');

# XGBoost
Existe un API que recubre la librería original para que tenga parámetros y métodos similares a Scikit-Learn. Los parámetros más relevantes son:

- `n_estimators` (int) – Número de árboles usados. Equivalente al número de rondas de boosting.
- `max_depth` (Optional[int]) – Máxima profundidad de los árboles.
- `subsample` (Optional[0-1]) - Proporción de submuestreo de las instancias de entrenamiento. Si se establece en 0.5, XGBoost muestreará aleatoriamente la mitad de los datos de entrenamiento antes de hacer crecer los árboles, lo que evitará el sobreajuste. El submuestreo se realizará una vez en cada iteración de boosting.
- `learning_rate` (Optional[float]) – Indice de aprendizaje en el Boosting  (xgb’s “eta”). Es un valor de regularización/penalización para evitar sobreajuste, limitando la influencia de cada modelo en el conjunto del ensemble
- `booster` (Optional[str]) – Especifica el modelo a utilizar: gbtree, gblinear or dart.
- `gamma` (Optional[float]) – (min_split_loss) Reducción mínima del error necesario para realizar otra partición en un nodo hoja del árbol.
- `grow_policy` – Política de crecimiento del árbol. 0: favorece la división en los nodos más cercanos al nodo, es decir, crece en profundidad. 1: favorece la división en los nodos con mayor cambio del error.


In [ ]:
!pip3 install xgboost

In [ ]:
# Instalación XGBoost: !pip install xgboost
from xgboost import XGBClassifier

In [ ]:
# en XGBoost no se pueden tener variables de salida categóricas,
# se deben codificar como enteros
# ==============================================================================

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

y_test_xg = le.transform(y_test)
y_train_xg = le.transform (y_train)

In [ ]:
# espacio de búsqueda
param_grid = {'max_depth'        : [None, 1, 3, 5, 10, 20],
              'subsample'        : [0.5, 1],
              'learning_rate'    : [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
              'n_estimators'     : sp_randint(50, 500)
              }

# Búsqueda por random search con validación cruzada
# ==============================================================================
inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
budget = 20
grid = RandomizedSearchCV(
    estimator  = XGBClassifier(random_state = 42,enable_categorical=True),
    param_distributions= param_grid,
    scoring    = 'balanced_accuracy',
    n_jobs     = multiprocessing.cpu_count() - 1,
    cv         = inner,
    refit      = True,
    verbose    = 0,
    n_iter=budget
)

grid.fit(X = X_train, y = y_train_xg)



In [ ]:
# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
modelo_final = grid.best_estimator_
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test_xg, y_test_pred)
print("Classification Report:",)
print (result)

# LightGBM
Se trata de un algoritmo de Boosting con rendimientos similares a XGBoost de forma más rápida. Parámetros más relevantes:
- `n_estimators` (int) – Número de árboles usados. Equivalente al número de rondas de boosting.
- `max_depth` (Optional[int]) – Máxima profundidad de los árboles. (<=0 indica que no hay límite)
- `subsample` (Optional[0-1]) - (o bagging_fraction) para especificar el porcentaje de muestras utilizadas por iteración de construcción del árbol. Esto significa que algunas filas se seleccionarán aleatoriamente para ajustar cada árbol. Esto mejora la generalización y  la velocidad de entrenamiento.
- `learning_rate` (Optional[float]) – Indice de aprendizaje en el Boosting. Es un valor de regularización/penalización para evitar sobreajuste, limitando la influencia de cada modelo en el conjunto del ensemble


In [ ]:
#!pip install lightgbm
#!conda install -c conda-forge lightgbm

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:

# Grid de hiperparámetros evaluados
# ==============================================================================
  
param_grid = {'num_leaves'       : [15, 31, 63],
              'max_depth'        : [-1, 10, 20, 30],
              'subsample'        : [0.5, 1],
              'learning_rate'    : [0.01, 0.1, 0.2],
              'n_estimators'     : [20, 40, 100, 200]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
budget = 10

# Set up the random search with cross-validation
grid = RandomizedSearchCV(LGBMClassifier(n_estimators=1000, random_state=42), 
                                   param_distributions=param_grid,
                                   n_iter=budget,
                                   scoring = 'balanced_accuracy', 
                                   cv=inner, 
                                   n_jobs = - 1,
                                   verbose=0,
                                   return_train_score = True, 
                                   random_state=42)

grid.fit(X = X_train, y = y_train)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head(4)

In [ ]:
grid.best_params_, grid.best_score_

In [ ]:
modelo_final = grid.best_estimator_
y_test_pred = modelo_final.predict(X_test)
result = metrics.classification_report(y_test, y_test_pred)
print("Classification Report:",)
print (result)

In [ ]:
modelo_final = grid.best_estimator_
# Entrenamos con todos los datos para el modelo final
_ = modelo_final.fit(X,y)